In [ ]:
import pandas as pd
import io
import requests
import datetime
import plotly.express as px

## Get the Timeseries Data

Add documentation about the time series here.

In [ ]:
from functions import get_time_series

confirmed, deaths, recovered, time_series_dates = get_time_series(local=True)

## Get all the Daily Reports 

Add documentation about these reports here.

In [ ]:
from functions import get_daily_reports

daily_report_data, daily_report_dates = get_daily_reports(local=True)

## All Done Gathering Data.

Let's merge the time_series_data into one dataframe

In [ ]:
from model import data_by_area
from data import us_confirmed 

data = data_by_area(area='Nebraska', col='State', df=us_confirmed)
data.plot()

In [ ]:

def data_by_area(area='US', col='Country/Region', df=None):
	from data import time_series_date_list
	print(col)
	data =pd.Series(
		[df.loc[(df[col] == area)][date].sum() for date in time_series_date_list],
		)
	return data


In [ ]:
data_by_area(area='US', df=confirmed).plot()

In [1]:
from functions import make_country_labels, make_state_labels

from data import confirmed

data = confirmed.copy()
print(data.groupby('Country/Region').sum().iloc[:,-1].sort_values(ascending=False).index)


make_country_labels(data=confirmed)

Getting daily reports using local data
Index(['Vietnam', 'French Guiana', 'Croatia', 'Cruise Ship', 'Cuba', 'Cyprus',
       'Czechia', 'Denmark', 'Dominican Republic', 'Ecuador',
       ...
       'Mongolia', 'Morocco', 'Nepal', 'Netherlands', 'New Zealand', 'Nigeria',
       'North Macedonia', 'Norway', 'Oman', 'Afghanistan'],
      dtype='object', name='Country/Region', length=125)


[{'label': 'Global', 'value': 'Global'},
 {'label': 'Vietnam', 'value': 'Vietnam'},
 {'label': 'French Guiana', 'value': 'French Guiana'},
 {'label': 'Croatia', 'value': 'Croatia'},
 {'label': 'Cruise Ship', 'value': 'Cruise Ship'},
 {'label': 'Cuba', 'value': 'Cuba'},
 {'label': 'Cyprus', 'value': 'Cyprus'},
 {'label': 'Czechia', 'value': 'Czechia'},
 {'label': 'Denmark', 'value': 'Denmark'},
 {'label': 'Dominican Republic', 'value': 'Dominican Republic'},
 {'label': 'Ecuador', 'value': 'Ecuador'},
 {'label': 'Egypt', 'value': 'Egypt'},
 {'label': 'Estonia', 'value': 'Estonia'},
 {'label': 'Ethiopia', 'value': 'Ethiopia'},
 {'label': 'Finland', 'value': 'Finland'},
 {'label': 'France', 'value': 'France'},
 {'label': 'Georgia', 'value': 'Georgia'},
 {'label': 'Israel', 'value': 'Israel'},
 {'label': 'Germany', 'value': 'Germany'},
 {'label': 'Greece', 'value': 'Greece'},
 {'label': 'Guadeloupe', 'value': 'Guadeloupe'},
 {'label': 'Guinea', 'value': 'Guinea'},
 {'label': 'Guyana', 'valu

In [12]:
data.groupby('Country/Region').sum().iloc[:,-2].sort_values(ascending=False)

Country/Region
China            80945
Italy            17660
Iran             11364
Korea, South      7979
Spain             5232
                 ...  
Mongolia             1
Guyana               1
Nepal                1
Cote d'Ivoire        1
Guadeloupe           1
Name: 3/13/20, Length: 125, dtype: int64

In [37]:
import pandas as pd
def get_daily_reports(local=True):
    start_date = pd.to_datetime('01/22/2020')
    end_date = pd.to_datetime('today')
    dates = pd.date_range(start_date, end_date)
    daily_reports = {}
    all_reports = []
    if local == True:
        print('Getting daily reports using local data')
        for date in dates:
            date_str = date.strftime('%m-%d-%Y')
            file_name = 'data/' + date_str + '.csv'
            try:
                df = pd.read_csv(file_name, header=0)
                df['Date'] = date_str
                all_reports.append(df)
            except:
                print("Failed to load {file}".format(file=file_name))
    elif local == False:
        print('Getting daily reports using on-line data')
        daily_report_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/'
        for date in dates:
            # File format: 'MM-DD-YYYY.csv'
            date_str = date.strftime('%m-%d-%Y')
            file_name =date_str + '.csv'
            url = daily_report_url + file_name
            try:
                df = pd.read_csv(file_name, header=0)
                df['Date'] = date_str
                all_reports.append(df)
            except:
                print("Failed to load {file}".format(file=file_name))
    daily_reports = pd.concat(all_reports, axis=0, ignore_index=True)
#    valid_dates = [pd.to_datetime(date) for date in list(daily_reports.keys())]
    return daily_reports

In [225]:
df = get_daily_reports()

df['Date']
df.Date = pd.to_datetime(df.Date)
#df['Last Update'] = pd.to_datetime(df['Last Update'])
#df['Last Update']

Getting daily reports using local data
Failed to load data/03-15-2020.csv


0       01-22-2020
1       01-22-2020
2       01-22-2020
3       01-22-2020
4       01-22-2020
           ...    
5627    03-14-2020
5628    03-14-2020
5629    03-14-2020
5630    03-14-2020
5631    03-14-2020
Name: Date, Length: 5632, dtype: object

In [79]:
print(df.columns)
area = 'US'
most_recent = df[df['Country/Region'] == area]['Last Update'].max() - pd.Timedelta('12 hours')
df[(df['Country/Region'] == area) & (df['Last Update'] >= most_recent)].sort_values('Last Update', ascending=False)

Index(['Province/State', 'Country/Region', 'Last Update', 'Confirmed',
       'Deaths', 'Recovered', 'Date', 'Latitude', 'Longitude'],
      dtype='object')


,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Date,Latitude,Longitude
5564,Vermont,US,2020-03-14 22:33:04,5.0,0.0,0.0,2020-03-14,44.0459,-72.7107
5492,Tennessee,US,2020-03-14 22:33:04,32.0,0.0,0.0,2020-03-14,35.7478,-86.6923
5468,Illinois,US,2020-03-14 22:33:04,64.0,0.0,2.0,2020-03-14,40.3495,-88.9861
5549,Kansas,US,2020-03-14 22:33:03,8.0,1.0,0.0,2020-03-14,38.5266,-96.7265
5509,Connecticut,US,2020-03-14 22:33:03,22.0,0.0,0.0,2020-03-14,41.5978,-72.7554
5516,South Carolina,US,2020-03-14 22:33:03,19.0,0.0,0.0,2020-03-14,33.8569,-80.9450
5533,Arkansas,US,2020-03-14 22:33:03,12.0,0.0,0.0,2020-03-14,34.9697,-92.3731
5504,Maryland,US,2020-03-14 22:33:03,26.0,0.0,0.0,2020-03-14,39.0639,-76.8021
5591,Idaho,US,2020-03-14 22:33:03,2.0,0.0,0.0,2020-03-14,44.2405,-114.4788
5499,Wisconsin,US,2020-03-14 22:33:03,27.0,0.0,1.0,2020-03-14,44.2685,-89.6165


In [87]:
def recent_updates(area='US', timedelta='12 hours', sort='Confirmed', df=None):
    ascending = {
        'Province/State': True
    }
    most_recent = df[df['Country/Region'] == area]['Last Update'].max() - pd.Timedelta(timedelta)
    return df[
        (df['Country/Region'] == area) & (df['Last Update'] >= most_recent)].sort_values(
            sort, ascending=ascending.get(sort, False))


In [90]:
import dash_table

In [221]:
from model import data_by_area
from data import time_series_dates, confirmed

all_countries = confirmed.groupby('Country/Region').sum()

pandemic = 100
pandemic_countries = all_countries[all_countries['3/13/20'] >= pandemic].copy()

#confirmed[confirmed.columns[4:]].diff(axis=1)/confirmed[confirmed.columns[4:]].plot()

ndays = 7
data = pandemic_countries[pandemic_countries.columns[-ndays:-1]].copy()
data = data.sort_values(list(data.columns)[-1:], ascending=False)

rates = (data.diff(axis=1)/data)
rates = rates[rates.columns[1:]]
rates = rates.sort_values(list(rates.columns)[-1:], ascending=False)

from math import log
doubling_times = log(2)/rates
doubling_times[doubling_times == np.inf]=None
print(doubling_times.mean(axis=1).sort_values())


Country/Region
Bahrain             -2.143999
Slovenia             1.689852
Denmark              1.877073
Spain                1.921839
Portugal             2.106765
Finland              2.253692
Austria              2.793679
Qatar                2.879694
Brazil               2.888313
Sweden               3.065609
Norway               3.088415
Netherlands          3.261701
US                   3.337406
Israel               3.342052
Iraq                 3.403773
Switzerland          3.585856
United Kingdom       3.702097
Australia            3.747705
Iceland              4.003622
Greece               4.055006
Italy                4.206927
Belgium              4.267430
France               4.324769
Germany              4.517577
Malaysia             4.991382
Iran                 6.757680
Czechia              6.975371
Singapore            8.082049
Canada               8.967479
Japan               15.145521
Korea, South        60.146161
China             2931.353063
Cruise Ship              

In [161]:
import numpy as np
import pandas as pd
df = pd.DataFrame(np.random.randint(0,10,size=(10, 9)), columns=list('ABCDEFGHI'))
#print(df)
#print(df.diff(axis=1))
df[df.columns[-1:]]

,I
0,2
1,1
2,6
3,5
4,5
5,0
6,2
7,1
8,2
9,4
